패키지 설치

In [0]:
!pip install mxnet-cu100
!pip install gluonnlp
!pip install gluoncv

In [0]:
!nvidia-smi

Fri Aug  9 02:04:46 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    17W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import mxnet as mx
from mxnet import nd, gluon, autograd
from mxnet.gluon import nn
import gluoncv
import gluonnlp as nlp

In [0]:
mx.__version__

'1.5.0'

In [0]:
gluoncv.__version__

'0.4.0'

In [0]:
nlp.__version__

'0.8.0'

In [0]:
## Setting transform
def transform(data, label):
    data = data.astype('float32')/255
    return data, label

train_dataset = mx.gluon.data.vision.datasets.FashionMNIST(train=True, transform=transform)
valid_dataset = mx.gluon.data.vision.datasets.FashionMNIST(train=False, transform=transform)

In [0]:
batch_size = 32
train_data_loader = mx.gluon.data.DataLoader(train_dataset, batch_size, shuffle=True, num_workers=1)
valid_data_loader = mx.gluon.data.DataLoader(valid_dataset, batch_size, num_workers=1)

DNN Model

In [0]:
model = nn.HybridSequential()
model.add(nn.Dense(128, activation='relu'))
model.add(nn.Dense(64, activation='relu'))
model.add(nn.Dense(10, activation='relu'))

In [0]:
model

HybridSequential(
  (0): Dense(None -> 128, Activation(relu))
  (1): Dense(None -> 64, Activation(relu))
  (2): Dense(None -> 10, Activation(relu))
)

In [0]:
ctx = mx.gpu()

In [0]:
## 모델에 대한 초기화 작업을 반드시 해주어야 함
model.initialize(mx.init.Xavier(), ctx=ctx)
model.hybridize()

In [0]:
# define loss and trainer.
criterion = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(model.collect_params(), 'sgd', {'learning_rate': 0.1})

In [0]:
epochs = 10
for epoch in range(epochs):
    # training loop (with autograd and trainer steps, etc.)
    cumulative_train_loss = mx.nd.zeros(1, ctx=ctx)
    training_samples = 0
    
    for batch_idx, (data, label) in enumerate(train_data_loader):
        data = data.as_in_context(ctx).reshape((-1, 784)) # 28*28=784
        label = label.as_in_context(ctx)
        with autograd.record():
            output = model(data)
            loss = criterion(output, label)
        loss.backward()
        trainer.step(data.shape[0])
        cumulative_train_loss += loss.sum()
        training_samples += data.shape[0]
    train_loss = cumulative_train_loss.asscalar()/training_samples

    # validation loop
    cumulative_valid_loss = mx.nd.zeros(1, ctx)
    valid_samples = 0
    for batch_idx, (data, label) in enumerate(valid_data_loader):
        data = data.as_in_context(ctx).reshape((-1, 784)) # 28*28=784
        label = label.as_in_context(ctx)
        output = model(data)
        loss = criterion(output, label)
        cumulative_valid_loss += loss.sum()
        valid_samples += data.shape[0]
    valid_loss = cumulative_valid_loss.asscalar()/valid_samples

    print("Epoch {}, training loss: {:.2f}, validation loss: {:.2f}".format(epoch+1, train_loss, valid_loss))


Epoch 1, training loss: 0.75, validation loss: 0.44
Epoch 2, training loss: 0.41, validation loss: 0.38
Epoch 3, training loss: 0.36, validation loss: 0.41
Epoch 4, training loss: 0.34, validation loss: 0.35
Epoch 5, training loss: 0.32, validation loss: 0.35
Epoch 6, training loss: 0.30, validation loss: 0.32
Epoch 7, training loss: 0.29, validation loss: 0.32
Epoch 8, training loss: 0.28, validation loss: 0.31
Epoch 9, training loss: 0.27, validation loss: 0.33
Epoch 10, training loss: 0.26, validation loss: 0.31


CNN Model

In [0]:
model_cnn = nn.HybridSequential()
model_cnn.add(nn.Conv2D(3, 3, 1, activation='relu'))
model_cnn.add(nn.Conv2D(3, 3, 1, activation='relu'))
model_cnn.add(nn.Flatten())
model_cnn.add(nn.Dense(100, activation='relu'))
model_cnn.add(nn.Dense(10, activation='relu'))

In [0]:
model_cnn

HybridSequential(
  (0): Conv2D(None -> 3, kernel_size=(3, 3), stride=(1, 1), Activation(relu))
  (1): Conv2D(None -> 3, kernel_size=(3, 3), stride=(1, 1), Activation(relu))
  (2): Flatten
  (3): Dense(None -> 100, Activation(relu))
  (4): Dense(None -> 10, Activation(relu))
)

In [0]:
model_cnn.initialize(mx.init.Xavier(), ctx=ctx)
model_cnn.hybridize()

In [0]:
# define loss and trainer.
criterion = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(model_cnn.collect_params(), 'sgd', {'learning_rate': 0.01})

In [0]:
epochs = 10
for epoch in range(epochs):
    # training loop (with autograd and trainer steps, etc.)
    cumulative_train_loss = mx.nd.zeros(1, ctx=ctx)
    training_samples = 0
    
    for batch_idx, (data, label) in enumerate(train_data_loader):
        data = data.as_in_context(ctx).transpose((0,3,1,2))
        label = label.as_in_context(ctx)
        with autograd.record():
            output = model_cnn(data)
            loss = criterion(output, label)
        loss.backward()
        trainer.step(data.shape[0])
        cumulative_train_loss += loss.sum()
        training_samples += data.shape[0]
    train_loss = cumulative_train_loss.asscalar()/training_samples

    # validation loop
    cumulative_valid_loss = mx.nd.zeros(1, ctx)
    valid_samples = 0
    for batch_idx, (data, label) in enumerate(valid_data_loader):
        data = data.as_in_context(ctx).transpose((0,3,1,2))
        label = label.as_in_context(ctx)
        output = model_cnn(data)
        loss = criterion(output, label)
        cumulative_valid_loss += loss.sum()
        valid_samples += data.shape[0]
    valid_loss = cumulative_valid_loss.asscalar()/valid_samples

    print("Epoch {}, training loss: {:.2f}, validation loss: {:.2f}".format(epoch+1, train_loss, valid_loss))


Epoch 1, training loss: 0.95, validation loss: 0.51
Epoch 2, training loss: 0.47, validation loss: 0.46
Epoch 3, training loss: 0.41, validation loss: 0.40
Epoch 4, training loss: 0.38, validation loss: 0.38
Epoch 5, training loss: 0.36, validation loss: 0.36
Epoch 6, training loss: 0.34, validation loss: 0.35
Epoch 7, training loss: 0.33, validation loss: 0.35
Epoch 8, training loss: 0.32, validation loss: 0.33
Epoch 9, training loss: 0.30, validation loss: 0.33
Epoch 10, training loss: 0.29, validation loss: 0.32
